In [2]:
%reset -f

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()

climate_model = 'CCSM4'

Mounted at /content/drive


In [ ]:
#Future Windows MEAN
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013],[2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = maca_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()


def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('pr')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', climate_model))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))

    def masking_clipping_fn(imobj):
      nonzeroMask_im = ee.Image(imobj).clip(area_shp).gt(0)
      nonzeroMasked_im = ee.Image(imobj).updateMask(nonzeroMask_im)
      return nonzeroMasked_im

    masked_ic = month_ic.map(masking_clipping_fn)
    mean_im = masked_ic.reduce(ee.Reducer.mean())
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))
  eimage = eimage.setDefaultProjection('EPSG:4326', transform)
  eimage = eimage.resample('bilinear').reproject(crs=proj.crs(), crsTransform=transform_new)

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='mean_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future Windows SDEV
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = maca_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()


def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('pr')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', climate_model))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))

    def masking_clipping_fn(imobj):
      nonzeroMask_im = ee.Image(imobj).clip(area_shp).gt(0)
      nonzeroMasked_im = ee.Image(imobj).updateMask(nonzeroMask_im)
      return nonzeroMasked_im

    masked_ic = month_ic.map(masking_clipping_fn)
    mean_im = masked_ic.reduce(ee.Reducer.stdDev())
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))
  eimage = eimage.setDefaultProjection('EPSG:4326', transform)
  eimage = eimage.resample('bilinear').reproject(crs=proj.crs(), crsTransform=transform_new)

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='sdev_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future Windows SKEW
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = maca_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()


def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('pr')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', climate_model))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))

    def masking_clipping_fn(imobj):
      nonzeroMask_im = ee.Image(imobj).clip(area_shp).gt(0)
      nonzeroMasked_im = ee.Image(imobj).updateMask(nonzeroMask_im)
      return nonzeroMasked_im

    masked_ic = month_ic.map(masking_clipping_fn)
    mean_im = masked_ic.reduce(ee.Reducer.skew())
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))
  eimage = eimage.setDefaultProjection('EPSG:4326', transform)
  eimage = eimage.resample('bilinear').reproject(crs=proj.crs(), crsTransform=transform_new)

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='skew_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future TMAX SDEV
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = maca_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()


def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('tasmax')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', climate_model))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    mean_im = month_ic.reduce(ee.Reducer.stdDev()).subtract(273.15)
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))
  eimage = eimage.setDefaultProjection('EPSG:4326', transform)
  eimage = eimage.resample('bilinear').reproject(crs=proj.crs(), crsTransform=transform_new)

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='txsd_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future TMIN SDEV
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = maca_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()


def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('tasmin')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', climate_model))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    mean_im = month_ic.reduce(ee.Reducer.stdDev()).subtract(273.15)
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))
  eimage = eimage.setDefaultProjection('EPSG:4326', transform)
  eimage = eimage.resample('bilinear').reproject(crs=proj.crs(), crsTransform=transform_new)

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='tnsd_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future SRAD
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = maca_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()


def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('rsds')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', climate_model))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    mean_im = month_ic.reduce(ee.Reducer.mean())
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))
  eimage = eimage.setDefaultProjection('EPSG:4326', transform)
  eimage = eimage.resample('bilinear').reproject(crs=proj.crs(), crsTransform=transform_new)

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='srad_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future SRAD SDEV
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = maca_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()


def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)
  year_ic = maca_ic.filterDate(start, end).select('rsds')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', climate_model))

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_ic = year_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    mean_im = month_ic.reduce(ee.Reducer.stdDev())
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))
  eimage = eimage.setDefaultProjection('EPSG:4326', transform)
  eimage = eimage.resample('bilinear').reproject(crs=proj.crs(), crsTransform=transform_new)

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='srsd_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()

  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1




In [ ]:
#Future TDEW CALCULATED (NOT AVAILABLE FOR CCSM4)
import ee

ee.Initialize(project='ee-andrewfullhart')

maca_ic = ee.ImageCollection('IDAHO_EPSCOR/MACAv2_METDATA')

nex_ic = ee.ImageCollection('NASA/NEX-DCP30')

first_im = nex_ic.first()
scale = first_im.projection().nominalScale().getInfo()

order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

year_windows = ee.List([[1974, 2013], [2000, 2029], [2010, 2039], [2020, 2049], [2030, 2059],
                        [2040, 2069], [2050, 2079], [2060, 2089], [2070, 2099]])

area_shp = ee.Geometry.Rectangle([-121, 30, -102, 43], 'EPSG:4326', False)

im = maca_ic.first()
proj = im.projection().getInfo()

transform = [
  proj['transform'][0],
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  proj['transform'][4],
  proj['transform'][5],
]

transform_new = [
  0.0083333333,
  proj['transform'][1],
  proj['transform'][2],
  proj['transform'][3],
  0.0083333333,
  proj['transform'][5],
]

proj = im.projection()


def year_fn(yrwinobj):
  start_year = ee.Number(ee.List(yrwinobj).get(0))
  end_year = ee.Number(ee.List(yrwinobj).get(1))
  start = ee.Date.fromYMD(ee.Number(start_year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(end_year).add(1), 1, 1)

  year_ic = maca_ic.filterDate(start, end)
  year_ic = year_ic.filter(ee.Filter.eq('model', climate_model))

  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))

  year_ic = year_ic.filter(modelfilter)
  year_rmax_ic = year_ic.select('rhsmax')
  year_rmin_ic = year_ic.select('rhsmin')
  year_tmax_ic = year_ic.select('tasmax')
  year_tmin_ic = year_ic.select('tasmin')

  def month_fn(mobj):
    m = ee.Number(mobj)
    month_rmax_ic = year_rmax_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    month_rmax_ic_list = month_rmax_ic.toList(month_rmax_ic.size())
    month_rmin_ic = year_rmin_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    month_rmin_ic_list = month_rmin_ic.toList(month_rmin_ic.size())
    month_tmax_ic = year_tmax_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    month_tmax_ic_list = month_tmax_ic.toList(month_tmax_ic.size())
    month_tmin_ic = year_tmin_ic.filter(ee.Filter.calendarRange(m, m, 'month'))
    month_tmin_ic_list = month_tmin_ic.toList(month_tmin_ic.size())

    def day_fn(dayiobj):
      day_i = ee.Number(dayiobj)
      rmax_im = ee.Image(month_rmax_ic_list.get(day_i))
      rmin_im = ee.Image(month_rmin_ic_list.get(day_i))
      ravg_im = ee.Image(rmax_im.add(rmin_im)).divide(2.)
      tmax_im = ee.Image(month_tmax_ic_list.get(day_i))
      tmin_im = ee.Image(month_tmin_ic_list.get(day_i))
      tavg_im = ee.Image(tmax_im.add(tmin_im)).divide(2.)
      tdew_im = tavg_im.subtract((ravg_im.multiply(-1.).add(100.)).divide(5))
      return tdew_im

    day_ic = ee.ImageCollection(ee.List.sequence(0, month_rmax_ic.size().subtract(1)).map(day_fn))
    mean_im = day_ic.reduce(ee.Reducer.mean()).subtract(273.15)
    return mean_im

  mean_ic_list = order_months.map(month_fn)
  return mean_ic_list

nested_mean_ic_list = year_windows.map(year_fn)
mean_ic_list = nested_mean_ic_list.flatten()


yrwin_i = 0
mo_i = 0
for i in range(12*year_windows.size().getInfo()):
  mo_i += 1
  start_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(0)).getInfo()
  end_year = ee.Number(ee.List(year_windows.get(yrwin_i)).get(1)).getInfo()

  eimage = ee.Image(mean_ic_list.get(i))
  eimage = eimage.setDefaultProjection('EPSG:4326', transform)
  eimage = eimage.resample('bilinear').reproject(crs=proj.crs(), crsTransform=transform_new)

  task = ee.batch.Export.image.toDrive(
          image=eimage,
          description='tdew_{}_{}_{}'.format(str(start_year), str(end_year), str(mo_i)),
          folder='GEE_Downloads',
          region=area_shp,
          scale=scale,
          crs='EPSG:4326',
          maxPixels=1e13)

  task.start()


  if mo_i == 12:
    mo_i = 0
    yrwin_i += 1


